# Importing libraries


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import requests

In [ ]:
import helper

In [ ]:
import factors
from factors import *

In [ ]:
import importlib
importlib.reload(helper)
importlib.reload(factors)

# Data Collection and cleaning

In [ ]:
# basic variables

start_date = '2000-01-01'
end_date = '2023-12-31' 
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

In [ ]:
nse_tickers = pd.read_csv("C:/Users/satya/Downloads/ind_nifty500list (1).csv")['Symbol']
nse_tickers = nse_tickers.to_list()
for count in range(len(nse_tickers)):
    nse_tickers[count] = nse_tickers[count] + ".NS"
nse_tickers

In [ ]:
price_data = yf.download(nse_tickers , start_date, end_date)
price_data = price_data['Adj Close']
price_data = price_data.dropna(axis=1, how='all')

In [ ]:
nse_stocks_data = clean_data(price_data, date_range)

In [ ]:
symbols = list(nse_stocks_data.keys())

In [ ]:
period = 'W'
time = 'training'

# Calculations

In [ ]:
returns_df = calculate_returns(nse_stocks_data, rebalance=period)

In [ ]:
returns_df

In [ ]:
if time == 'training':
    returns_df = returns_df.loc[:, (returns_df.columns > '2005')& (returns_df.columns < '2019')]
elif time == 'testing':
    returns_df = returns_df.loc[:,returns_df.columns > '2019']

In [ ]:
'''
A dictionary to store the return%. 
The strategy name is the key and the return-df is value
'''
strategy_returns_ls = {}

In [ ]:
months = [12]
pct = [50]

In [ ]:
for m in months:
    momentum = calculate_m_momentum(nse_stocks_data, m)
     
    for p in pct:
        strategy = f'{m}_momentum_{p}'
        momentum_long(nse_stocks_data, momentum, returns_df, p,  strategy= strategy, strategy_returns=strategy_returns)

### Momentum

In [ ]:
for m in months:
    momentum = calculate_m_momentum(nse_stocks_data, m, resample=period)
    # if time == 'training':
    #     momentum = momentum.loc[:, (momentum.columns > '2005') & (momentum.columns < '2019')]
    # elif time == 'testing':
    #     momentum = momentum.loc[:,momentum.columns > '2019']
    for p in pct:
        strategy = f'{m}_momentum_ls_{p}'
        momentum_long_sort(nse_stocks_data, momentum, returns_df, p, p, strategy, strategy_returns=strategy_returns_ls)
        


In [ ]:
strategy_returns_ls['12_momentum_ls_50'].rename({0:'12_momentum_ls_50'}, axis=1, inplace=True)

### 12-1 momentum

In [ ]:
momentum_df = calculate_12_1_momentum(nse_stocks_data, resample=period)

In [ ]:
if time == 'training':
    returns_df = returns_df.loc[:, (returns_df.columns > '2005')& (returns_df.columns < '2019')]
elif time == 'testing':
    momentum_df = momentum_df.loc[:, momentum_df.columns > '2019']

In [ ]:
for p_top in pct:
    momentum_12_1_ls(nse_stocks_data, momentum_df, returns_df, p_top, strategy_returns=strategy_returns_ls)

In [ ]:
strategy_returns_ls['12_1_momentum_ls_50'].rename({0:'12_1_momentum_ls_50'}, axis=1, inplace=True)

### REVERSION

In [ ]:
momentum_rev = calculate_m_momentum(nse_stocks_data, 1, resample = period)

In [ ]:
momentum_rev

In [ ]:
if time == 'training':
    momentum_rev = momentum_rev.loc[:, (momentum_rev.columns>'2005') & (momentum_rev.columns<'2019')]
elif time == 'testing':
    momentum_rev = momentum_rev.loc[:, momentum_rev.columns>'2019']

In [ ]:
for p_long in pct:
        strategy = f'Reversal_ls_{p_long}'
        reversion_long_sort(nse_stocks_data, momentum_rev, returns_df, p_long, p_long, strategy=strategy, strategy_returns=strategy_returns_ls)
            

In [ ]:
strategy_returns_ls['Reversal_ls_50']

### SEASONALITY

3 years seasonality

In [ ]:
strategy = f'3_yr_seasonality_2_ls'
seasonality_ls(stocks_data=nse_stocks_data, return_df=returns_df, strategy_returns=strategy_returns_ls, rebalance=period, y=3, m = 2, n=2, strategy= strategy, )

In [ ]:
(sorted(strategy_returns_ls['3_yr_seasonality_2_ls'].items()))

5-years

In [ ]:
times = [3]
for m in times:
    strategy = f'5_yr_seasonality_{m}_ls'
    seasonality_ls(nse_stocks_data, returns_df, strategy_returns=strategy_returns_ls, rebalance=period, y=5, m = m, n=m, strategy= strategy, )

In [ ]:
dict(sorted(strategy_returns_ls['3_yr_seasonality_2_ls'].items()))

### VALUE

In [ ]:
strategy = 'Value_15_85'
price_strategy_ls(nse_stocks_data, returns_df, 15, 85, strategy, strategy_returns=strategy_returns_ls)

In [ ]:
strategy = 'Value_25_75'
price_strategy_ls(nse_stocks_data, returns_df, 75, 25, strategy, strategy_returns=strategy_returns_ls)

In [ ]:
strategy_returns_ls['Value_15_85'] = strategy_returns_ls['Value_15_85'][strategy_returns_ls['Value_15_85'].index > '2006']

In [ ]:
strategy_returns_ls['Value_15_85']

### VOL

In [ ]:
def calculate_vol(return_data, y=1):
    volatility = return_data.T.rolling(window=52*y).std()
    volatility = volatility * np.sqrt(52)
    
    return volatility.T.dropna(axis=1, how='all')

In [ ]:
vol_df = calculate_vol(returns_df)
top = top_p_percentage_stocks(vol_df, 33)
btm = bottom_p_percentage_stocks(vol_df, 33)
ret = portfolio_return(returns_df, top, btm)

In [ ]:
strategy_returns_ls['1_year_vol_ls_33']  = pd.DataFrame.from_dict(dict(sorted(ret.items())), orient='index', columns=['1_year_vol_ls_33'])

In [ ]:
vol_df = calculate_vol(returns_df)
top = top_p_percentage_stocks(vol_df, 50)
btm = bottom_p_percentage_stocks(vol_df, 50)
ret = portfolio_return(returns_df, top, btm)
strategy_returns_ls['1_year_vol_ls_50']  = pd.DataFrame.from_dict(dict(sorted(ret.items())), orient='index', columns=['1_year_vol_ls_50'])

In [ ]:
strategy_returns_ls['1_year_vol_ls_50'] = strategy_returns_ls['1_year_vol_ls_50'][strategy_returns_ls['1_year_vol_ls_50'].index > '2006']

In [ ]:
strategy_returns_ls['1_year_vol_ls_50']

# METRICES

In [ ]:
for key in strategy_returns_ls.keys():
    print(calculate_overall_cagrs(strategy_returns_ls[key], period))
    print(calculate_overall_sharpe_ratios(strategy_returns_ls[key], period))
    print('-----------------------------------------------------------')
#     strategy_returns_ls[key].sort_index()
# # Join the dataframes based on their date index
# strategies = pd.concat(strategy_returns_ls.values(), axis=1)

In [ ]:
for strategy, df in strategy_returns_ls.items():
    df.columns = [f'{strategy}_{col}' for col in df.columns]

# Combine all DataFrames into a single DataFrame
df = pd.concat(strategy_returns_ls.values(), axis=1)

In [ ]:
df

In [ ]:
cagrs = calculate_overall_cagrs(df, 'W')

In [ ]:
sr = calculate_overall_sharpe_ratios(df, 'W')

# Plots

In [ ]:
import plotly.graph_objs as go
import plotly.express as px

def calculate_cumulative_returns(df):
    cumulative_returns_df = (1 + df / 100).cumprod()
    return cumulative_returns_df

def plot_strategy_performance(strategy_df):
    # Ensure the index is a datetime index
    strategy_df.index = pd.to_datetime(strategy_df.index)
    
    # Calculate cumulative returns
    cumulative_returns_df = calculate_cumulative_returns(strategy_df)
    
    # Create the plot
    fig = go.Figure()

    # Add traces for each strategy
    for strategy in cumulative_returns_df.columns:
        fig.add_trace(go.Scatter(
            x=cumulative_returns_df.index,
            y=cumulative_returns_df[strategy],
            mode='lines',
            name=strategy
        ))
    
    # Add titles and labels
    fig.update_layout(
        title='Cumulative Returns of Strategies',
        xaxis_title='Date',
        yaxis_title='Cumulative Return',
        legend_title='Strategies'
    )

    # Show the plot
    fig.show()

plot_strategy_performance(df)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def calculate_cumulative_returns(df):
    """Calculate cumulative returns from percentage returns."""
    cumulative_returns_df = (1 + df / 100).cumprod()
    return cumulative_returns_df

def plot_strategy_performance(strategy_df):
    # Ensure the index is a datetime index
    strategy_df.index = pd.to_datetime(strategy_df.index)
    
    # Sort the DataFrame by the index (dates)
    strategy_df = strategy_df.sort_index()
    
    # Calculate cumulative returns
    cumulative_returns_df = calculate_cumulative_returns(strategy_df)
    
    # Plot each strategy individually
    for strategy in cumulative_returns_df.columns:
        plt.figure(figsize=(12, 8))
        
        # Plot cumulative returns
        plt.plot(cumulative_returns_df.index, cumulative_returns_df[strategy], label=strategy)
        
        # Annotate the final cumulative return
        final_return = cumulative_returns_df[strategy].iloc[-1]
        plt.text(cumulative_returns_df.index[-1], final_return, f'{final_return:.2%}', 
                 fontsize=12, ha='left', va='center')
        
        # Only show left and bottom axes
        plt.gca().spines['top'].set_visible(False)
        plt.gca().spines['right'].set_visible(False)
        
        # Set axis labels and title
        plt.xlabel('Date', fontsize=14)
        plt.ylabel('Cumulative Return', fontsize=14)
        plt.title('Cumulative Return ', fontsize=16)
                
        # Display the plot
        plt.show()



In [ ]:
df.fillna(0, inplace=True)

In [ ]:
plot_strategy_performance(df)